In [1]:
# 기본 패키지 불러오기 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
plt.style.use("seaborn")
sns.set(font_scale=1)
sns.set_style("whitegrid")

# 그래프를 노트북 안에 그리기 위해 설정
%matplotlib inline

# 맥 OS 폰트 깨짐 방지 코드
from matplotlib import rc
import matplotlib as mpl 
import matplotlib.font_manager as fm 

rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

## Inside Airbnb: London Data

### 데이터 불러오기

In [2]:
# 데이터 불러오기
london_lists_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/listings.csv')
london_lists_summary_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/listings_outline.csv')
london_neighbourhoods_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/neighbourhoods.csv')
london_reviews_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/reviews.csv')
london_reviews_summary_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/reviews_outline.csv')

# 분석할 df 별도 복사
london_lists = london_lists_raw.copy()
london_reviews = london_reviews_raw.copy()

# 불러온 df 확인
london_lists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90852 entries, 0 to 90851
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            90852 non-null  int64  
 1   listing_url                                   90852 non-null  object 
 2   scrape_id                                     90852 non-null  int64  
 3   last_scraped                                  90852 non-null  object 
 4   source                                        90852 non-null  object 
 5   name                                          90852 non-null  object 
 6   description                                   87851 non-null  object 
 7   neighborhood_overview                         47521 non-null  object 
 8   picture_url                                   90842 non-null  object 
 9   host_id                                       90852 non-null 

### 분석에서 사용하지 않을 컬럼 삭제

In [103]:
# 1차 선별한 컬럼 리스트 
# 1차 선별한 컬럼 27개 + 검증만 하고 삭제하기로 합의한 컬럼 3개('host_has_profile_pic','host_identity_verified', 'has_availability')
# 총 30개 컬럼 선별 

columns_selected = ['id', 'host_id', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_total_listings_count', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood_cleansed', \
    'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'has_availability', 'number_of_reviews', 'number_of_reviews_l30d', 'review_scores_rating', 'review_scores_accuracy', \
    'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']

# 선별한 컬럼만 적용
london_lists = london_lists[columns_selected]

# url도 포함한 df (실제 에어비앤비 사이트 들어가서 확인할 용도)
columns_selected_url = ['id', 'listing_url', 'host_id', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_total_listings_count', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood_cleansed', \
    'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'has_availability', 'number_of_reviews', 'number_of_reviews_l30d', 'review_scores_rating', 'review_scores_accuracy', \
    'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']

london_url = london_lists_raw[columns_selected_url]

# 변수 정리 
london_lists = london_lists.rename(columns={
    'id': '숙소_id',
    'host_id': '호스트_id',
    'host_response_time': '답변_평균시간',
    'host_response_rate': '문의_응답률',
    'host_acceptance_rate': '예약_수락률',
    'host_is_superhost': '슈퍼호스트',
    'host_total_listings_count': '숙소_수',
    'host_has_profile_pic' : '프로필_사진',
    'host_identity_verified' : '호스트_신원',
    'neighbourhood_cleansed': '숙소_지역',
    'room_type': '숙소_유형',
    'accommodates': '수용_인원수',
    'bathrooms': '욕실수',
    'bedrooms': '침실수',
    'beds': '침대수',
    'amenities': '편의시설',
    'price': '숙소_가격',
    'minimum_nights': '최소_숙박일',
    'maximum_nights': '최대_숙박일',
    'has_availability' : '예약_가능여부',
    'number_of_reviews': '리뷰수',
    'number_of_reviews_l30d': '30일_리뷰수',
    'review_scores_rating': '리뷰점수',
    'review_scores_accuracy': '숙소_정확성_리뷰점수',
    'review_scores_cleanliness': '숙소_청결도_리뷰점수',
    'review_scores_checkin': '숙소_체크인_리뷰점수',
    'review_scores_communication': '숙소_소통_리뷰점수',
    'review_scores_location': '숙소_위치_리뷰점수',
    'review_scores_value': '숙소_가격_리뷰점수',
    'reviews_per_month': '평균_리뷰수'
})

# url 포함 df 변수 정리 
london_url = london_url.rename(columns={
    'id': '숙소_id',
    'listing_url' : '숙소_url',
    'host_id': '호스트_id',
    'host_response_time': '답변_평균시간',
    'host_response_rate': '문의_응답률',
    'host_acceptance_rate': '예약_수락률',
    'host_is_superhost': '슈퍼호스트',
    'host_total_listings_count': '숙소_수',
    'host_has_profile_pic' : '프로필_사진',
    'host_identity_verified' : '호스트_신원',
    'neighbourhood_cleansed': '숙소_지역',
    'room_type': '숙소_유형',
    'accommodates': '수용_인원수',
    'bathrooms': '욕실수',
    'bedrooms': '침실수',
    'beds': '침대수',
    'amenities': '편의시설',
    'price': '숙소_가격',
    'minimum_nights': '최소_숙박일',
    'maximum_nights': '최대_숙박일',
    'has_availability' : '예약_가능여부',
    'number_of_reviews': '리뷰수',
    'number_of_reviews_l30d': '30일_리뷰수',
    'review_scores_rating': '리뷰점수',
    'review_scores_accuracy': '숙소_정확성_리뷰점수',
    'review_scores_cleanliness': '숙소_청결도_리뷰점수',
    'review_scores_checkin': '숙소_체크인_리뷰점수',
    'review_scores_communication': '숙소_소통_리뷰점수',
    'review_scores_location': '숙소_위치_리뷰점수',
    'review_scores_value': '숙소_가격_리뷰점수',
    'reviews_per_month': '평균_리뷰수'
})


# 컬럼명 확인
london_lists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90852 entries, 0 to 90851
Data columns (total 30 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   숙소_id        90852 non-null  int64  
 1   호스트_id       90852 non-null  int64  
 2   답변_평균시간      57797 non-null  object 
 3   문의_응답률       57797 non-null  object 
 4   예약_수락률       65213 non-null  object 
 5   슈퍼호스트        90463 non-null  object 
 6   숙소_수         90847 non-null  float64
 7   프로필_사진       90847 non-null  object 
 8   호스트_신원       90847 non-null  object 
 9   숙소_지역        90852 non-null  object 
 10  숙소_유형        90852 non-null  object 
 11  수용_인원수       90852 non-null  int64  
 12  욕실수          57894 non-null  float64
 13  침실수          78118 non-null  float64
 14  침대수          57837 non-null  float64
 15  편의시설         90852 non-null  object 
 16  숙소_가격        57885 non-null  object 
 17  최소_숙박일       90852 non-null  int64  
 18  최대_숙박일       90852 non-null  int64  
 19  예약_가

### 결측치 처리에 앞서서 리뷰가 0인 컬럼 제거

In [104]:
# 리뷰수가 0인 컬럼 조건 
condition_review_0 = london_lists['리뷰수'] == 0

# 별도의 DF로 저장
london_lists_review_0 = london_lists[condition_review_0]

리뷰가 0인 데이터의 특징 파악

In [105]:
# 리뷰수가 0인 컬럼의 특징 파악_리뷰데이터 관련

# 개수: 24,983개
print('리뷰가 0개인 런던 내 숙소 개수: ')
print(london_lists_review_0.shape)
print()

# 리뷰가 0개인데 각 리뷰점수가 Null값이 아닌 숙소 개수 

score_type = ['리뷰점수', '숙소_정확성_리뷰점수', '숙소_청결도_리뷰점수', '숙소_체크인_리뷰점수', '숙소_소통_리뷰점수', '숙소_위치_리뷰점수', '숙소_가격_리뷰점수', '평균_리뷰수']

for type in score_type:
    print(f'{type} :')
    print(f'{type}가 null이 아닌 데이터의 개수 >> {london_lists_review_0[type].notnull().sum()}개')
    print()

리뷰가 0개인 런던 내 숙소 개수: 
(24983, 30)

리뷰점수 :
리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_정확성_리뷰점수 :
숙소_정확성_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_청결도_리뷰점수 :
숙소_청결도_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_체크인_리뷰점수 :
숙소_체크인_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_소통_리뷰점수 :
숙소_소통_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_위치_리뷰점수 :
숙소_위치_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_가격_리뷰점수 :
숙소_가격_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

평균_리뷰수 :
평균_리뷰수가 null이 아닌 데이터의 개수 >> 0개



In [108]:
# # 리뷰수가 0인 컬럼의 특징 파악_슈퍼호스트 관련

print(london_lists_review_0['슈퍼호스트'].unique())
print()
print('리뷰슈가 0개인 숙소 중 호스트가 슈퍼호스트인지 여부: ')
print(london_lists_review_0['슈퍼호스트'].value_counts())
print()
print('리뷰수가 0개인 숙소 데이터 중 슈퍼호스트가 null값인 데이터 개수:')
print(f'{london_lists_review_0["슈퍼호스트"].isnull().sum()}개')

['f' 't' nan]

리뷰슈가 0개인 숙소 중 호스트가 슈퍼호스트인지 여부: 
f    23754
t     1171
Name: 슈퍼호스트, dtype: int64

리뷰수가 0개인 숙소 데이터 중 슈퍼호스트가 null값인 데이터 개수:
58개


리뷰가 0개가 아닌 데이터만 사용

In [110]:
# 리뷰 개수가 0이 아닌 컬럼 조건 설정
condition_review = london_lists['리뷰수'] != 0

# 리뷰 개수가 0이 아닌 컬럼으로 df 업데이트 
london_lists = london_lists[condition_review]
london_url = london_url[condition_review]

# print(london_lists[:10])
# print()
print(london_lists.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65869 entries, 0 to 90735
Data columns (total 30 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   숙소_id        65869 non-null  int64  
 1   호스트_id       65869 non-null  int64  
 2   답변_평균시간      45111 non-null  object 
 3   문의_응답률       45111 non-null  object 
 4   예약_수락률       51189 non-null  object 
 5   슈퍼호스트        65538 non-null  object 
 6   숙소_수         65869 non-null  float64
 7   프로필_사진       65869 non-null  object 
 8   호스트_신원       65869 non-null  object 
 9   숙소_지역        65869 non-null  object 
 10  숙소_유형        65869 non-null  object 
 11  수용_인원수       65869 non-null  int64  
 12  욕실수          43758 non-null  float64
 13  침실수          58052 non-null  float64
 14  침대수          43703 non-null  float64
 15  편의시설         65869 non-null  object 
 16  숙소_가격        43728 non-null  object 
 17  최소_숙박일       65869 non-null  int64  
 18  최대_숙박일       65869 non-null  int64  
 19  예약_가

/var/folders/_j/yl1g13yd0_s25zcx_fwnjdkm0000gn/T/ipykernel_10047/1749373705.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  london_url = london_url[condition_review]


In [111]:
# 문의_응답률 / 예약_수략률 >> 타입변경

london_lists['문의_응답률'] = london_lists['문의_응답률'].str.rstrip('%').astype('float')
london_lists['예약_수락률'] = london_lists['예약_수락률'].str.rstrip('%').astype('float')

# 기초 통계 요약
london_lists[['문의_응답률', '예약_수락률']].describe()

,문의_응답률,예약_수락률
count,45111.000000,51189.000000
mean,94.720334,85.460333
std,16.803026,23.021839
min,0.000000,0.000000
25%,100.000000,80.000000
50%,100.000000,97.000000
75%,100.000000,100.000000
max,100.000000,100.000000


결측치때문에 제대로 된 수치가 나오지 않은 것으로 추정

# 결측치

결측치 처리에 집중해야할 컬럼 선택

In [112]:
# 컬럼 전체 결측치 확인 

london_lists.isnull().sum()
# 전체 개수 : 65,869개 중 

숙소_id              0
호스트_id             0
답변_평균시간        20758
문의_응답률         20758
예약_수락률         14680
슈퍼호스트            331
숙소_수               0
프로필_사진             0
호스트_신원             0
숙소_지역              0
숙소_유형              0
수용_인원수             0
욕실수            22111
침실수             7817
침대수            22166
편의시설               0
숙소_가격          22141
최소_숙박일             0
최대_숙박일             0
예약_가능여부         1333
리뷰수                0
30일_리뷰수            0
리뷰점수               0
숙소_정확성_리뷰점수       31
숙소_청결도_리뷰점수       26
숙소_체크인_리뷰점수       61
숙소_소통_리뷰점수        36
숙소_위치_리뷰점수        61
숙소_가격_리뷰점수        63
평균_리뷰수             0
dtype: int64

### 결측치가 천단위 이상인  컬럼

1. '답변_평균시간', '문의_응답률', '예약_수락률'
* 호스트 커뮤니케이션 관련 컬럼 > 고객의 만족도(좋은 숙소?)에 큰 영향을 미치는 요인 중 하나 
2. '욕실수', '침실수', '침대수' 
*  숙소 가격을 결정하는데 큰 요소가 될 수 있는 카테고리 > 크롤링으로 데이터 값 채워넣기(값이 시간에 영향을 받지 않음)
3. '숙소_가격' : 크롤링으로 채워넣기가 힘듬. 시간에 따른 변동성이 큰 값.
* 숙소 가격에 영향을 미칠 요인 추측 : 위치, 숙소 크기, 숙소 형태(ex. 아파트?) 

### 결측치가 천단위 이하인 컬럼

1. 슈퍼호스트 
2. 예약_가능여부 
3. '숙소_정확성_리뷰점수', '숙소_청결도_리뷰점수', '숙소_체크인_리뷰점수', '숙소_소통_리뷰점수', '숙소_위치_리뷰점수', '숙소_가격_리뷰점수'

# 슈퍼호스트

슈퍼호스트 기준
1. 숙박 10건 이상 호스팅 또는 3건의 예약에 걸쳐 총 100박 이상 호스팅
2. 응답률 90% 이상 유지
3. 예약 취소율 1% 미만 유지. 단, 정상참작이 가능한 상황에 따른 예약 취소는 제외
4. 전체 평점 4.8점 이상 유지(후기 작성 기한인 14일이 지나거나, 그전에라도 게스트와 호스트 양측이 모두 후기를 제출하면 후기는 슈퍼호스트 실적에 계산됩니다).

In [116]:
# 슈퍼호스트 null 값 : 331개
print(f'슈퍼호스트 null값 개수: {london_lists["슈퍼호스트"].isnull().sum()}개')

# 슈퍼호스트 null값의 비율: 0.5% >> 삭제해도 무방 
print(f'슈퍼호스트 null값의 비율: {round((london_lists["슈퍼호스트"].isnull().sum() / 65869) * 100, 2)}%')

슈퍼호스트 null값 개수: 331개
슈퍼호스트 null값의 비율: 0.5%


In [114]:
london_lists['슈퍼호스트'].value_counts()

f    52458
t    13080
Name: 슈퍼호스트, dtype: int64

In [87]:
# 슈퍼호스트 null값인 데이터 특징 파악

condition_superhost_null = london_lists['슈퍼호스트'].isnull()
london_lists[condition_superhost_null][['문의_응답률', '예약_수락률', '숙소_수', '리뷰수', '리뷰점수', '평균_리뷰수']].describe()

,숙소_수,리뷰수,리뷰점수,평균_리뷰수
count,331.000000,331.000000,331.000000,331.000000
mean,26.957704,35.525680,4.675921,1.490242
std,28.076271,50.373229,0.320133,1.390061
min,2.000000,1.000000,3.400000,0.010000
25%,5.000000,4.000000,4.500000,0.380000
50%,14.000000,14.000000,4.730000,1.050000
75%,36.000000,48.500000,4.970000,2.155000
max,123.000000,342.000000,5.000000,6.860000


### 슈퍼호스트가 null값인 데이터 특징? 
1. 호스트가 여러 숙소를 운영하고 있다. (숙소수 중위수 14개, 미니멈 2개, q1 5개)
2. 리뷰수가 적다. (중위수 14개)
3. 예약 수략률과 문의 응답률이 모두 좋다 (호스트 단위 집계 항목)

실제 데이터를 좀 봐볼까?

In [64]:
london_url[condition_superhost_null][['숙소_url', '호스트_id', '문의_응답률', '예약_수락률', '숙소_수', '리뷰수', '리뷰점수', '평균_리뷰수']].sort_values(by = ['숙소_수', '리뷰수'], ascending=[False, True])[:30]

,숙소_url,호스트_id,문의_응답률,예약_수락률,숙소_수,리뷰수,리뷰점수,평균_리뷰수
78819,https://www.airbnb.com/rooms/1022410255586290392,60524679,98%,100%,123.0,4,4.25,1.05
30653,https://www.airbnb.com/rooms/34884624,60524679,98%,100%,123.0,6,4.33,0.10
12449,https://www.airbnb.com/rooms/14754559,60524679,98%,100%,123.0,147,4.41,1.61
13539,https://www.airbnb.com/rooms/16328145,60524679,98%,100%,123.0,154,4.38,1.76
13538,https://www.airbnb.com/rooms/16327915,60524679,98%,100%,123.0,158,4.42,1.79
11022,https://www.airbnb.com/rooms/13541121,60524679,98%,100%,123.0,159,4.48,1.70
12216,https://www.airbnb.com/rooms/14483249,60524679,98%,100%,123.0,162,4.43,1.75
11762,https://www.airbnb.com/rooms/14092256,60524679,98%,100%,123.0,164,4.38,1.79
10670,https://www.airbnb.com/rooms/13422389,60524679,98%,100%,123.0,167,4.45,1.77
12448,https://www.airbnb.com/rooms/14754229,60524679,98%,100%,123.0,191,4.45,2.09


엥? 호스트가 같은 값이 많네?

In [90]:
# 호스트가 겹치는 숙소(슈퍼호스트가 null값인) 숫자 세기 

# 조건 설정 
condition_result_host_1 = (london_lists[condition_superhost_null]['호스트_id'].value_counts() == 1)

# True와 False 값의 개수를 각각 세기
true_count = condition_result_host_1.sum()
false_count = (~condition_result_host_1).sum()

# 출력
print(f"겹치지 않은 호스트의 수: {true_count}")
print(f"겹치는 호스트의 수: {false_count}")

겹치지 않은 호스트의 수: 83
겹치는 호스트의 수: 47


In [91]:
london_lists[condition_superhost_null]['호스트_id'].value_counts()[:30]

494256171    21
239389535    15
268416645    14
154555136    11
4086714      11
30362264     10
127463633    10
329579576    10
60524679     10
183088797     9
157295299     8
23966399      7
67938882      6
456934029     6
68502358      5
423475816     5
509171821     5
232786181     5
190939015     5
107983837     4
196192499     4
401748044     4
35351106      4
462011865     3
144834468     3
97874184      3
13792962      3
235963197     3
8419096       3
267646946     3
Name: 호스트_id, dtype: int64

숙소 개수는 미니멈이 2개 였으니까 전체 데이터에서 슈퍼호스트가 null값인 호스트의 다른 숙소가 있나 한 번 확인해보자.

# 답변_평균시간

* '답변 평균시간'을 제외하고자 했던 논리가 슈퍼호스트 선정 기준이었던 것으로 기억함
* 근데 엄연히 답변 평균 시간과 응답률은 다른 속성 컬럼임으로 임의로 전처리를 진행해보기로 결정 

In [49]:
london_lists['답변_평균시간'].unique()

array(['within a few hours', 'within a day', 'within an hour', nan,
       'a few days or more'], dtype=object)

In [50]:
london_lists['답변_평균시간'].isnull().sum()

20758

In [51]:
london_lists['답변_평균시간'].value_counts()

within an hour        31207
within a few hours     7548
within a day           5008
a few days or more     1348
Name: 답변_평균시간, dtype: int64

### 답변_평균시간이 null인 값들의 슈퍼호스트 여부 

# 문의_응답률

In [52]:
# 문의_응답률 컬럼 

london_lists['문의_응답률'].isnull().sum()

20758

In [53]:
# 문의 응답률이 not null값인 숙소 >> 에어비앤비 사이트에서 샘플 확인

condition_response_notnull = london_lists['문의_응답률'].notnull()
london_lists_raw[condition_response_notnull][['listing_url', 'name', 'host_response_rate']][:30]

# >> 사이트 숙소 상세 설명 페이지 내 문의 응답률 위치 확인 

/var/folders/_j/yl1g13yd0_s25zcx_fwnjdkm0000gn/T/ipykernel_10047/2834148228.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  london_lists_raw[condition_response_notnull][['listing_url', 'name', 'host_response_rate']][:30]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

응답률이 나와있는 숙소도 현재 사이트 내 문의 응답률과 다른 경우도 있음 
* https://www.airbnb.com/rooms/170524
* 66% > 33 %

In [ ]:
# 문의 응답률이 null값인 숙소 >> 에어비앤비 사이트에서 샘플 확인

# 선별한 컬럼의 데이터프레임에서 문의응답률이 null 값인 조건 설정 
condition_response_null = london_lists['문의_응답률'].isnull()  

# raw 데이터프레임에서 문의응답률이 null 값인 조건 설정 
raw_response_null = london_lists_raw['host_response_rate'].isnull()
london_lists_raw[raw_response_null][['listing_url', 'name']]

,listing_url,name
5,https://www.airbnb.com/rooms/313710,Pretty Home Kensal Green NW10 1 bed
6,https://www.airbnb.com/rooms/168359,Stunning 2 Bed Apartment in Elephant & Castle
8,https://www.airbnb.com/rooms/24328,"Battersea live/work artist house, garden communal"
11,https://www.airbnb.com/rooms/170702,A Double Room with River Views
12,https://www.airbnb.com/rooms/33332,Beautiful Ensuite Richmond-upon-Thames borough
...,...,...
90844,https://www.airbnb.com/rooms/1115272705566417319,Primrose Hill Studio #29
90845,https://www.airbnb.com/rooms/1115272736222611647,Primrose Hill Studio #28
90846,https://www.airbnb.com/rooms/1115272737028206976,Primrose Hill Studio #26
90847,https://www.airbnb.com/rooms/1115272743905612628,Primrose Hill Studio #25


문의 응답률이 null값'인 숙소의 경우의 수 (가설)

1. 숙소 이용이 없어서 문의 또한 없기 때문에
2. 실제로 문의 응답을 안하는 경우 (카운팅을 하지 못하기 때문)

<br />

3. 에어비앤비 호스트 정보에 문의 응답률이 누락된 경우 
4. 데이터를 가져오면서 누락된 경우 및 기타 오류 

In [ ]:
# 문의 응답률이 null 값인 숙소의 리뷰수와 리뷰 점수 조회

london_lists[condition_response_null][['리뷰수', '리뷰점수']]
london_lists[condition_response_null][['리뷰수', '리뷰점수']].sort_values(by='리뷰수', ascending=False)

,리뷰수,리뷰점수
6874,711,4.87
885,575,4.95
1144,559,4.89
536,548,4.60
967,400,4.63
...,...,...
25321,0,NaN
25322,0,NaN
25334,0,NaN
25340,0,NaN


리뷰가 많은데 문의 응답이 없는 경우가 있네?
* 이 케이스가 어떤 케이스인지 실제 사이트에서 확인해보자

In [ ]:
london_lists_raw[raw_response_null][['listing_url', 'name']].shape

(33055, 2)

In [ ]:
london_lists_raw[raw_response_null][['listing_url', 'name', 'number_of_reviews', 'review_scores_rating']].sort_values(by='number_of_reviews', ascending=False)[:30]

,listing_url,name,number_of_reviews,review_scores_rating
6874,https://www.airbnb.com/rooms/9046849,"Near HEATHROW AIRPORT, London, double room.",711,4.87
885,https://www.airbnb.com/rooms/877864,Bright guest room in gay household,575,4.95
1144,https://www.airbnb.com/rooms/1296836,"Covent Garden Flat Listed Building, London.",559,4.89
536,https://www.airbnb.com/rooms/502190,Large Double room private bath near Kilburn tube,548,4.60
967,https://www.airbnb.com/rooms/756481,Brand new studio in Victoria 15A,400,4.63
4030,https://www.airbnb.com/rooms/5878880,INDEPENDENT COSY LITTLE HOUSE IN CENTRAL LONDON,397,4.86
20,https://www.airbnb.com/rooms/315658,Double Room nr Oval Vauxhall Tubes,389,4.75
1087,https://www.airbnb.com/rooms/1117720,Central London: Sunny single room!,369,4.75
8819,https://www.airbnb.com/rooms/11226035,Tranquil comfortable single bedroom/females only,357,4.91
13151,https://www.airbnb.com/rooms/15855571,"LHR,free travelzone,UB35BD vPrivate,vClean,Gar...",354,4.91


슈퍼호스트가 아닌 경우 문의 응답률이 호스트 정보에 나오질 않나? 
*  확인 필요

슈퍼호스트 컬럼 확인

In [ ]:
# 슈퍼호스트 컬럼 null값 확인 
london_lists_raw['host_is_superhost'].isnull().sum()

389

In [ ]:
# 슈퍼호스트 컬럼 유니크값 확인 
london_lists_raw['host_is_superhost'].unique()

array(['t', 'f', nan], dtype=object)

In [ ]:
# 슈퍼호스트 컬럼 값별 개수 확인
# 슈퍼호스트 값이 t : 14,251
# 슈퍼호스트 값이 f : 76,212

# 30개 컬럼 선별한 데이터프레임(london_lists)의 컨디션
condition_superhost_t = london_lists['슈퍼호스트'] == 't'
condition_superhost_f = london_lists['슈퍼호스트'] == 'f'

# raw 데이터프레임(london_lists_raw)의 컨디션
raw_superhost_t = london_lists_raw['host_is_superhost'] == 't'
raw_superhost_f = london_lists_raw['host_is_superhost'] == 'f'
print('슈퍼호스트인 숙소 ')
print(london_lists_raw[raw_superhost_t].shape)
print()
print('슈퍼호스트가 아닌 숙소')
print(london_lists_raw[raw_superhost_f].shape)

슈퍼호스트인 숙소 
(14251, 75)

슈퍼호스트가 아닌 숙소
(76212, 75)


* 문의응답률 null 값 개수 : 33,055 개
* 슈퍼호스트가 아닌 숙소 : 76,212 개

<br />

*  슈퍼호스트가 아닌 경우에도 응답률 수집이 가능하긴 함!

리뷰가 많은데 문의응답률이 null값인 숙소의 슈퍼호스트 여부 확인

In [ ]:
london_lists_raw[raw_response_null]['host_is_superhost'].unique()

array(['f', 't', nan], dtype=object)

In [ ]:
# '문의_응답률'이 null인 숙소를 '리뷰수' 내림차순으로 정렬

london_lists_raw[raw_response_null][['listing_url', 'name', 'number_of_reviews', 'review_scores_rating', 'host_is_superhost']].sort_values(by='number_of_reviews', ascending=False)[:50]

,listing_url,name,number_of_reviews,review_scores_rating,host_is_superhost
6874,https://www.airbnb.com/rooms/9046849,"Near HEATHROW AIRPORT, London, double room.",711,4.87,f
885,https://www.airbnb.com/rooms/877864,Bright guest room in gay household,575,4.95,f
1144,https://www.airbnb.com/rooms/1296836,"Covent Garden Flat Listed Building, London.",559,4.89,f
536,https://www.airbnb.com/rooms/502190,Large Double room private bath near Kilburn tube,548,4.60,f
967,https://www.airbnb.com/rooms/756481,Brand new studio in Victoria 15A,400,4.63,f
4030,https://www.airbnb.com/rooms/5878880,INDEPENDENT COSY LITTLE HOUSE IN CENTRAL LONDON,397,4.86,f
20,https://www.airbnb.com/rooms/315658,Double Room nr Oval Vauxhall Tubes,389,4.75,f
1087,https://www.airbnb.com/rooms/1117720,Central London: Sunny single room!,369,4.75,f
8819,https://www.airbnb.com/rooms/11226035,Tranquil comfortable single bedroom/females only,357,4.91,f
13151,https://www.airbnb.com/rooms/15855571,"LHR,free travelzone,UB35BD vPrivate,vClean,Gar...",354,4.91,f


* 응답률 null값 중 리뷰가 많은 숙소는 슈퍼호스트가 아니다.'는 가설은 기각. 
* 응답률 null값 중 리뷰가 많은 숙소의 대부분의 경우 슈퍼호스트가 아니다.' 이거는 맞는거 같은데? 

또 다른 가설: 응답률이 나와있지 않으면서 슈퍼호스트인 경우 >> 다른 숙소에서 슈퍼호스트를 달고 해당 숙소는 오픈한지 얼마되지 않은 것 아닐까 ?
1. 리뷰는 숙소 단위로 카운트되는게 맞으므로, 리뷰가 많을 경우엔 해당 가설도 해당되지 않음 
2. 리뷰가 얼마 없는데 슈퍼호스트인 경우는 해당될 수 있음 >> 이 경우엔 실제 사이트에 들어가서 리뷰가 남겨진 날짜를 확인하면 증명할 수 있을 듯 

In [ ]:
# '문의_응답률'이 null값이고 '슈퍼호스트'가 t/f인 조건 생성

raw_response_null_superhost_t = raw_response_null & (london_lists_raw['host_is_superhost'] == 't')
raw_response_null_superhost_f = raw_response_null & (london_lists_raw['host_is_superhost'] == 'f')

raw_response_null_superhost_t

0        False
1        False
2        False
3        False
4        False
         ...  
90847    False
90848    False
90849    False
90850    False
90851    False
Length: 90852, dtype: bool

In [ ]:
# '문의_응답률'이 null값이고 '슈퍼호스트'가 t인 숙소 조회 : 총 907개 

london_lists_raw[raw_response_null_superhost_t][['listing_url', 'name', 'number_of_reviews', 'review_scores_rating']].sort_values(by='number_of_reviews', ascending=False)

,listing_url,name,number_of_reviews,review_scores_rating
9876,https://www.airbnb.com/rooms/12444228,Room close to Central London - Wi-Fi and TV inc.,329,4.73
3560,https://www.airbnb.com/rooms/5041618,Pimlico Victoria Double Room Georgian Garden Flat,289,4.88
3276,https://www.airbnb.com/rooms/4772751,Stylish 2 Bedroom Flat Central London *ZONE 1*,288,4.61
29452,https://www.airbnb.com/rooms/33040646,Cosy room in the heart of the city!,219,4.80
5162,https://www.airbnb.com/rooms/7257296,VERY SAFE area for female travellers/Flexi wor...,209,4.96
...,...,...,...,...
34712,https://www.airbnb.com/rooms/39941435,Parkview Flat of Real Tranquility,0,NaN
60239,https://www.airbnb.com/rooms/838154978747603826,Mezzanine Double Room,0,NaN
71150,https://www.airbnb.com/rooms/953343244946347684,King bedroom+private shower room,0,NaN
63871,https://www.airbnb.com/rooms/884863856530629176,Entire flat in Notting Hill,0,NaN


In [ ]:
# '문의_응답률'이 null값이고 '슈퍼호스트'이면서 리뷰가 0개인 숙소 개수 : 37개 

london_lists_raw[raw_response_null_superhost_t & (london_lists_raw['number_of_reviews'] == 0)][['listing_url', 'name', 'number_of_reviews', 'review_scores_rating']].shape

(37, 4)

'문의_응답률'이 null값이고 '슈퍼호스트'이면서 리뷰가 0개인 숙소 개수 : 37개 

--> 37개의 숙소는 슈퍼호스트가 새로 오픈한 숙소가 맞을듯 

In [ ]:
london_lists_raw[raw_response_null_superhost_t & (london_lists_raw['number_of_reviews'] == 0)][['listing_url', 'name', 'number_of_reviews', 'review_scores_rating', 'host_is_superhost', 'host_response_rate']]

,listing_url,name,number_of_reviews,review_scores_rating,host_is_superhost,host_response_rate
11604,https://www.airbnb.com/rooms/13922077,Beautiful large room Highbury and Islington,0,NaN,t,NaN
14098,https://www.airbnb.com/rooms/16772532,North Acton two bed two bath,0,NaN,t,NaN
23837,https://www.airbnb.com/rooms/25768390,Perfect for Wimbledon Tennis,0,NaN,t,NaN
34712,https://www.airbnb.com/rooms/39941435,Parkview Flat of Real Tranquility,0,NaN,t,NaN
37028,https://www.airbnb.com/rooms/42733043,Spacious room females ONLY double bed London E18,0,NaN,t,NaN
37098,https://www.airbnb.com/rooms/42849943,Stylish flat overlooking Hackney Downs,0,NaN,t,NaN
41621,https://www.airbnb.com/rooms/50633813,Vacant Professional housing in West Drayton,0,NaN,t,NaN
43196,https://www.airbnb.com/rooms/52646658,The Rokxy King Suite,0,NaN,t,NaN
45441,https://www.airbnb.com/rooms/557769923114262393,Family Ensuite@SkylarkB&B,0,NaN,t,NaN
47925,https://www.airbnb.com/rooms/633402445806743860,Islington Family Home,0,NaN,t,NaN


* 1번째 숙소 : 슈퍼호스트가 아닌데 슈퍼호스트라고 나와있고.. 
* 2번째 숙소 : 슈퍼호스트는 맞는데 문의_응답률이 나와있고... 
* 3번째 숙소 : 슈퍼호스트는 아닌데 문의_응답률이 나와있고... 
* 4번째 숙소 : 슈퍼호스트는 맞는데 문의_응답률이 나와있고... 
* 5번째 숙소 : 이 숙소는 링크 진입 자체가 안되네??

* 우선 응답률 null 값을 처리하는 방법은 투 트랙으로 가는 것이 좋을 듯.
    1. 리뷰 수가 많은 숙소의 경우 평균 또는 중위수로 처리
    2. 리뷰 수가 없는 숙소의 경우는 삭제하는 것이 맞지 않을까? 

근데 리뷰가 없는 애들은 분석할게 없으니까 먼저 정리하는게 맞지 않을까? 

우선은 중위수랑 평균 먼저 봐보자.

In [ ]:
# 문의 응답률 >> 타입 변경 >> float

london_lists['문의_응답률'] = london_lists['문의_응답률'].str.rstrip('%').astype('float')
london_lists['문의_응답률'].head()

0    100.0
1     86.0
2    100.0
3    100.0
4    100.0
Name: 문의_응답률, dtype: float64

In [ ]:
london_lists['문의_응답률'].describe()

count    57797.000000
mean        91.930360
std         21.931639
min          0.000000
25%         98.000000
50%        100.000000
75%        100.000000
max        100.000000
Name: 문의_응답률, dtype: float64

응답률 > 호스트 단위로 측정됨 (에어비앤비 CS 문의)

슈퍼호스트인데 문의 응답률이 없는 경우는 무엇인가? (에어비앤비 CS 팀에서 답변은 어렵다고 함)

슈퍼호스트  > 호스트 아이디로 묶어서 숙소 별로 일치하는지도 확인해야
